In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/non-performing-assets-npa/npa_dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/non-performing-assets-npa/npa_dataset.csv')

#find missing count column wise
df.isnull().sum() 

Loan_ID              0
Customer_ID          0
Loan_Amount          0
Loan_Type            0
Credit_Score         0
Repayment_History    0
Collateral_Value     0
Loan_Tenure          0
Default_Status       0
dtype: int64

In [22]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300221 entries, 0 to 300220
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Loan_ID            300221 non-null  object 
 1   Customer_ID        300221 non-null  object 
 2   Loan_Amount        300221 non-null  int64  
 3   Loan_Type          300221 non-null  object 
 4   Credit_Score       300221 non-null  int64  
 5   Repayment_History  300221 non-null  float64
 6   Collateral_Value   300221 non-null  int64  
 7   Loan_Tenure        300221 non-null  int64  
 8   Default_Status     300221 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 20.6+ MB


Index(['Loan_ID', 'Customer_ID', 'Loan_Amount', 'Loan_Type', 'Credit_Score',
       'Repayment_History', 'Collateral_Value', 'Loan_Tenure',
       'Default_Status'],
      dtype='object')

In [8]:
df.describe()

,Loan_Amount,Credit_Score,Repayment_History,Collateral_Value,Loan_Tenure,Default_Status
count,3.002210e+05,300221.000000,300221.000000,3.002210e+05,300221.000000,300221.000000
mean,2.530722e+06,575.282745,75.031481,1.992232e+06,185.535649,0.270927
std,1.427794e+06,159.167892,14.448399,1.153038e+06,100.431357,0.444439
min,5.000400e+04,300.000000,50.000042,1.800000e+01,12.000000,0.000000
25%,1.294031e+06,437.000000,62.489519,9.935240e+05,99.000000,0.000000
50%,2.535654e+06,575.000000,75.052078,1.988123e+06,185.000000,0.000000
75%,3.767650e+06,713.000000,87.555989,2.988275e+06,272.000000,1.000000
max,4.999999e+06,850.000000,99.999641,3.999995e+06,359.000000,1.000000


In [24]:
#Check duplicate rows
print(df.duplicated().sum())
print(df['Customer_ID'].duplicated().sum() )

0
0


In [21]:
# Percentage of Default: 1- NPA, 0-Standard
print((df['Default_Status'].value_counts(normalize=True)*100).round(2))

Default_Status
0    72.91
1    27.09
Name: proportion, dtype: float64


In [25]:
#avg credit score, avg_loan_amt, total loan amt  loan type and npa status wise
#df1 = df.groupby(['Loan_Type','Default_Status'])['Credit_Score'].mean().round().reset_index().rename(columns={'Credit_Score':'Avg_Credit_Score'})
df_out = df.groupby(['Loan_Type', 'Default_Status']).agg({
    'Loan_ID': 'count',
    'Credit_Score': 'mean',
    'Loan_Amount' : ['mean','sum']
}).reset_index()

#Rename GroupBY Columns
df_out.columns = ['Loan_Type', 'Default_Status',
               'Loan_Count', 'Avg_Credit_Score', 'Avg_Loan_Amt', 'Loan_Total']

df_out['Loan_Count'] = df_out['Loan_Count'].apply(lambda x: f"{x:,}")
df_out['Avg_Credit_Score'] = df_out['Avg_Credit_Score'].round().astype(int)
df_out['Avg_Loan_Amt'] = df_out['Avg_Loan_Amt'].round().astype(int).apply(lambda x: f"{x:,}")
df_out['Loan_Total'] = df_out['Loan_Total'].apply(lambda x: f"{x:,}")

df_out

,Loan_Type,Default_Status,Loan_Count,Avg_Credit_Score,Avg_Loan_Amt,Loan_Total
0,Business,0,"43,970",621,"2,525,545","111,048,235,524"
1,Business,1,"16,282",450,"2,520,069","41,031,757,193"
2,Education,0,"44,019",622,"2,531,246","111,422,915,849"
3,Education,1,"16,214",450,"2,544,153","41,250,904,830"
4,Home,0,"43,690",623,"2,534,903","110,749,923,525"
5,Home,1,"16,136",450,"2,540,403","40,991,938,982"
6,Personal,0,"43,520",622,"2,526,054","109,933,855,791"
7,Personal,1,"16,337",449,"2,530,354","41,338,388,138"
8,Vehicle,0,"43,684",621,"2,530,246","110,531,286,103"
9,Vehicle,1,"16,369",450,"2,533,857","41,476,710,774"


In [27]:
df.assign(
    Collateral_Coverage = df['Collateral_Value'] / df['Loan_Amount']
).groupby('Default_Status')['Collateral_Coverage'].describe()


,count,mean,std,min,25%,50%,75%,max
Default_Status,,,,,,,,
0,218883.0,1.861779,4.271495,0.000012,0.392401,0.787867,1.540001,77.413827
1,81338.0,1.848836,4.262776,0.000051,0.392814,0.786973,1.530706,76.885277


In [28]:
df.groupby('Default_Status')['Loan_Tenure'].describe()


,count,mean,std,min,25%,50%,75%,max
Default_Status,,,,,,,,
0,218883.0,185.302148,100.417731,12.0,98.0,185.0,272.0,359.0
1,81338.0,186.164007,100.465940,12.0,100.0,187.0,273.0,359.0


In [30]:
risk_by_type = df.groupby('Loan_Type').agg(
    Default_Rate=('Default_Status', 'mean'),
    Avg_Loan_Amount=('Loan_Amount', 'mean'),
    Total_Loan_Amount=('Loan_Amount', 'sum')
).reset_index()

risk_by_type['Default_Rate'] = (risk_by_type['Default_Rate'] * 100).round(2)
risk_by_type


,Loan_Type,Default_Rate,Avg_Loan_Amount,Total_Loan_Amount
0,Business,27.02,2.524065e+06,152079992717
1,Education,26.92,2.534721e+06,152673820679
2,Home,26.97,2.536387e+06,151741862507
3,Personal,27.29,2.527227e+06,151272243929
4,Vehicle,27.26,2.531231e+06,152007996877


In [45]:
top_loans = df.nlargest(10, 'Loan_Amount')[
    ['Loan_ID', 'Customer_ID', 'Loan_Amount', 'Loan_Type', 'Default_Status']
]
top_loans['Default_Status']=1

In [47]:
#Collateral and loss potential
df['Collateral_Coverage'] = df['Collateral_Value'] / df['Loan_Amount']

bins = [0, 0.5, 1, 1.5, 100]
labels = ['<50%','50–100%','100–150%','>150%']
df['Cov_Bucket'] = pd.cut(df['Collateral_Coverage'], bins=bins, labels=labels, include_lowest=True)

cov_view = df.groupby('Cov_Bucket').agg(
    Loan_Count=('Loan_ID','count'),
    Default_Rate=('Default_Status','mean'),
    Avg_Coverage=('Collateral_Coverage','mean'),
    Exposure=('Loan_Amount','sum')
).reset_index()

cov_view['Default_Rate_%'] = (cov_view['Default_Rate'] * 100).round(2)
cov_view


/tmp/ipykernel_55/114049781.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cov_view = df.groupby('Cov_Bucket').agg(


,Cov_Bucket,Loan_Count,Default_Rate,Avg_Coverage,Exposure,Default_Rate_%
0,<50%,95662,0.271215,0.250325,319584872688,27.12
1,50–100%,87205,0.270420,0.734380,280006750826,27.04
2,100–150%,40393,0.273884,1.217222,89420337078,27.39
3,>150%,76961,0.269591,5.466887,70763956117,26.96
